# Finding patterns in categorical attributes

In [1]:
# pip install mlxtend

In [10]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

from prefixspan import PrefixSpan

In [12]:
df = pd.read_csv('../data/all_preprocessed.csv')
df.describe()

,author,content,poem name,age,type,age_type
count,573,573,571,573,573,573
unique,67,506,508,2,3,6
top,william shakespeare,originally published in poetry march 1914,Canto IV,Renaissance,Love,Renaissance Love
freq,71,4,3,315,326,243


In [13]:
data = df[['author', 'age', 'type']]
data

,author,age,type
0,william shakespeare,Renaissance,Mythology & Folklore
1,duchess of newcastle margaret cavendish,Renaissance,Mythology & Folklore
2,thomas bastard,Renaissance,Mythology & Folklore
3,edmund spenser,Renaissance,Mythology & Folklore
4,richard barnfield,Renaissance,Mythology & Folklore
...,...,...,...
568,sara teasdale,Modern,Love
569,hart crane,Modern,Love
570,william butler yeats,Modern,Love
571,carl sandburg,Modern,Love


In [14]:
transactions = data.values.tolist()
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
df_encoded.head()

,Love,Modern,Mythology & Folklore,Nature,Renaissance,archibald macleish,asil bunting,carl sandburg,christopher marlowe,conrad aiken,...,t s eliot,thomas bastard,thomas campion,thomas heywood,thomas lodge,thomas nashe,wallace stevens,william butler yeats,william byrd,william shakespeare
0,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
frequentItemsets = apriori(df_encoded, min_support=0.1, use_colnames=True)

In [16]:
rules = association_rules(frequentItemsets, metric="confidence", min_threshold=0.7)

In [17]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Love),(Renaissance),0.568935,0.549738,0.424084,0.745399,1.355916,0.111318,1.768498,0.608937
1,(Renaissance),(Love),0.549738,0.568935,0.424084,0.771429,1.355916,0.111318,1.885908,0.582974
2,(Nature),(Modern),0.328098,0.450262,0.235602,0.718085,1.594817,0.087872,1.950015,0.555094
3,(william shakespeare),(Renaissance),0.123909,0.549738,0.123909,1.000000,1.819048,0.055792,inf,0.513944
